# Sparse Conv Feature

In [1]:
from voxelize import *
import os
from tqdm import tqdm
# data_path = '/home/jiazx_ug/dataset/S4/O/training/velodyne/335422.bin'
# label_path = '/home/jiazx_ug/dataset/S4/O/training/label_2/335422.txt'

data_dir = '/home/jiazx_ug/dataset/S4/O/training/velodyne/'
label_dir = '/home/jiazx_ug/dataset/S4/O/training/label_2/'

idx_list = [str(data.split('.')[0]) for data in os.listdir(data_dir)]
idx_list.sort()

In [2]:
label_dict = {'Car':0, 'Pedestrian':1, 'Rider':2, 'Truck':3, 'Van':4}
vsize_xyz = np.array([0.5, 0.5, 1])       
coors_range_xyz = np.array([-70.4, -40, -3, 70.4, 40, 1])  
num_point_features = 4                                 
max_num_points_per_voxel = 5               
max_num_voxels = 16000
input_channels = 4

grid_size = get_grid_size(coors_range_xyz, vsize_xyz)
backbone = input_backbone(input_channels, grid_size)
checker = isPointInQuadrangle()

In [4]:
all_features = []
all_labels = []

for idx in tqdm(idx_list):
    data_path = data_dir + idx + '.bin'
    label_path = label_dir + idx + '.txt'
    
    points = read_data(data_path)
    voxels, voxel_indices, num_points = create_voxel(points, vsize_xyz, coors_range_xyz,num_point_features ,max_num_points_per_voxel, max_num_voxels)
    input_feature, input_indices = get_input_feature(voxels, num_points, voxel_indices, backbone)
    indices_3d = indices_to_3d(input_indices, grid_size)

    corners_3Ds, label_obj = read_label(label_path)
    corners_voxelaxis = corners_to_voxelaxis(corners_3Ds, coors_range_xyz, vsize_xyz)
    feature_keys = get_inner_voxel(corners_voxelaxis, checker)
    features, labels = get_inner_voxel_feature(input_feature, indices_3d, feature_keys, grid_size, label_obj)
    all_features += features
    all_labels += labels
    
    #tqdm.write(f"IDX {idx} with {len(features)}, all {len(all_features)}")


100%|██████████| 11938/11938 [12:15<00:00, 16.23it/s]


In [5]:
feature_array = np.array(all_features)
feature_array.shape

(5687309, 16)

# MRMR

In [7]:
import pandas as pd
from mrmr import mrmr_classif
feature_array = pd.DataFrame(feature_array)
feature_label = pd.Series(all_labels)
feature_array.shape, feature_label.shape

((5687309, 16), (5687309,))

In [14]:
selected_features = mrmr_classif(X=feature_array, y=feature_label, K=3)
selected_features

100%|██████████| 3/3 [00:06<00:00,  2.33s/it]


[5, 8, 3]

In [33]:
feature_array.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,0.0,0.938805,1.180978,0.274972,1.323238,0.0,1.026129,0.193213,0.694095,0.000000,0.396268,0.056021,0.0,0.975139,0.0
1,0.0,0.0,0.938805,1.180978,0.274972,1.323238,0.0,1.026129,0.193213,0.694095,0.000000,0.396268,0.056021,0.0,0.975139,0.0
2,0.0,0.0,1.214502,0.702635,1.910038,0.516538,0.0,0.000000,1.961679,1.532110,1.901384,1.848315,1.955724,0.0,0.000000,0.0
3,0.0,0.0,0.938805,1.180978,0.274972,1.323238,0.0,1.026129,0.193213,0.694095,0.000000,0.396268,0.056021,0.0,0.975139,0.0
4,0.0,0.0,0.938805,1.180978,0.274972,1.323238,0.0,1.026129,0.193213,0.694095,0.000000,0.396268,0.056021,0.0,0.975139,0.0


In [1]:
# visualize the selected features in 3d, color by label
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

idx = np.random.choice(len(feature_array), len(feature_array)//1000)
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(feature_array[selected_features[0]].iloc[idx], feature_array[selected_features[1]].iloc[idx], feature_array[selected_features[2]].iloc[idx], c=feature_label.iloc[idx])
plt.show()


NameError: name 'np' is not defined

# Selection